#Import Library

In [38]:
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import datetime as dt
!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud

import nltk
nltk.download('all')

import csv
import requests
from io import StringIO

import numpy as np
!pip install gensim scikit-learn --quiet
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

#Loading Dataset

In [39]:
data = pd.read_csv('Grab_app_reviews.csv')

print(data.columns)
print(data.head())

jumlah_ulasan, jumlah_kolom = data.shape
print(f'Jumlah Ulasan: {jumlah_ulasan}, Jumlah Kolom: {jumlah_kolom}')

print(data.info())

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')
                               reviewId         userName  \
0  16b5e7fa-1229-419a-be97-29c1d8e74c9d  Pengguna Google   
1  5284e97c-1b5d-4dd3-99b7-05da0d611de8  Pengguna Google   
2  6600edc8-16c2-4ef2-9f45-b1dd4e9d0790  Pengguna Google   
3  9515e78e-1aa7-427c-b308-c5c86f60d28d  Pengguna Google   
4  2451a450-479f-413c-8139-e4eb5b5c4b9a  Pengguna Google   

                                           userImage  \
0  https://play-lh.googleusercontent.com/EGemoI2N...   
1  https://play-lh.googleusercontent.com/EGemoI2N...   
2  https://play-lh.googleusercontent.com/EGemoI2N...   
3  https://play-lh.googleusercontent.com/EGemoI2N...   
4  https://play-lh.googleusercontent.com/EGemoI2N...   

                                             content  score  thumbsUpCount  \
0  Di edit jadi bintang 1 dulu ya

In [40]:
#Hapus data hilang dan duplikat
clean_data = data.dropna()
clean_data = clean_data.drop_duplicates()
jumlah_ulasan_cleaned, jumlah_kolom_cleaned = clean_data.shape

print(clean_data.shape)

print(f'Jumlah Ulasan: {jumlah_ulasan_cleaned}, Jumlah Kolom: {jumlah_kolom_cleaned}')
print(clean_data.info())

(21637, 11)
Jumlah Ulasan: 21637, Jumlah Kolom: 11
<class 'pandas.core.frame.DataFrame'>
Index: 21637 entries, 0 to 107999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              21637 non-null  object
 1   userName              21637 non-null  object
 2   userImage             21637 non-null  object
 3   content               21637 non-null  object
 4   score                 21637 non-null  int64 
 5   thumbsUpCount         21637 non-null  int64 
 6   reviewCreatedVersion  21637 non-null  object
 7   at                    21637 non-null  object
 8   replyContent          21637 non-null  object
 9   repliedAt             21637 non-null  object
 10  appVersion            21637 non-null  object
dtypes: int64(2), object(9)
memory usage: 2.0+ MB
None


#Preprocessing Text

In [41]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text):
    text = text.lower()
    return text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [42]:
slangwords = {"@": "di",
              "abis": "habis",
              "wtb": "beli",
              "masi": "masih",
              "wts": "jual",
              "wtt": "tukar",
              "bgt": "banget",
              "maks": "maksimal",
              "bru": "baru",
              "dpt" : "dapat",
              "skrg": "sekarang",
              "jgn": "jangan",
              "yg": "yang"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [43]:
clean_data['text_clean'] = clean_data['content'].apply(cleaningText)

clean_data['text_casefoldingText'] = clean_data['text_clean'].apply(casefoldingText)

clean_data['text_slangwords'] = clean_data['text_casefoldingText'].apply(fix_slangwords)

clean_data['text_tokenizingText'] = clean_data['text_slangwords'].apply(tokenizingText)

clean_data['text_stopword'] = clean_data['text_tokenizingText'].apply(filteringText)

clean_data['text_akhir'] = clean_data['text_stopword'].apply(toSentence)

In [44]:
print(clean_data)

                                    reviewId              userName  \
0       16b5e7fa-1229-419a-be97-29c1d8e74c9d       Pengguna Google   
3       9515e78e-1aa7-427c-b308-c5c86f60d28d       Pengguna Google   
4       2451a450-479f-413c-8139-e4eb5b5c4b9a       Pengguna Google   
6       b0a8706e-72a9-448a-aaf9-90e6d7b8fdb6       Pengguna Google   
7       159a18b8-3402-478c-9fcc-3b5c76b1c97c       Pengguna Google   
...                                      ...                   ...   
107973  a96c0f23-ae9a-4008-a16a-b3a23280ff2f          jeje huraeji   
107986  9d10a5c9-add8-4eab-87cf-a113eabf1605      Oktavia Khaerani   
107994  9cc17cce-aab9-43c7-a398-9c349a2fb150  Sailor off the bes't   
107997  0b572784-f7ec-490a-9c0d-e7b9576335a1       DAFA PRODUCTION   
107999  48d242d9-9f37-4f24-bc84-300fee15066f          Yuda saputra   

                                                userImage  \
0       https://play-lh.googleusercontent.com/EGemoI2N...   
3       https://play-lh.googleusercon

#Pelabelan

In [45]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")

In [46]:
# Fungsi untuk menentukan polaritas sentimen
def sentiment_analysis_lexicon_indonesia(text):

    score = 0

    for word in text:

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if (score >= 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'

    return score, polarity

results = clean_data['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_data['polarity_score'] = results[0]
clean_data['polarity'] = results[1]
print(clean_data['polarity'].value_counts())
print(clean_data.columns)

polarity
negative    11942
positive     9695
Name: count, dtype: int64
Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'text_clean', 'text_casefoldingText',
       'text_slangwords', 'text_tokenizingText', 'text_stopword', 'text_akhir',
       'polarity_score', 'polarity'],
      dtype='object')


# Eksplorasi Label

## Data Splitting

In [47]:
text_col = 'text_akhir'
label_col = 'polarity'

# Bersihkan teks sederhana
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

clean_data[text_col] = clean_data[text_col].apply(clean_text)

# Encode label ke bentuk numerik
le = LabelEncoder()
clean_data['label_encoded'] = le.fit_transform(clean_data[label_col])


## Skema 1

In [48]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    clean_data[text_col], clean_data['label_encoded'], test_size=0.2, random_state=42, stratify=clean_data['label_encoded'])

# TF-IDF + SVM
tfidf = TfidfVectorizer(max_features=5000)
X_train1_tfidf = tfidf.fit_transform(X_train1)
X_test1_tfidf = tfidf.transform(X_test1)

svm_model = SVC()
svm_model.fit(X_train1_tfidf, y_train1)

y_pred1 = svm_model.predict(X_test1_tfidf)

report = classification_report(y_test1, y_pred1, target_names=le.classes_, output_dict=True)

report_df = pd.DataFrame(report).transpose()

for col in ['precision', 'recall', 'f1-score']:
  report_df[col] = (report_df[col]*100).round(2).astype(str) + '%'

print("Skema 1: SVM + TF-IDF + 80:20")
print(report_df[['precision', 'recall', 'f1-score']])

Skema 1: SVM + TF-IDF + 80:20
             precision  recall f1-score
negative        90.56%  92.76%   91.65%
positive         90.8%  88.09%   89.42%
accuracy        90.67%  90.67%   90.67%
macro avg       90.68%  90.42%   90.54%
weighted avg    90.67%  90.67%   90.65%


## Skema 2

In [52]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    clean_data[text_col], clean_data['label_encoded'], test_size=0.3, random_state=42, stratify=clean_data['label_encoded'])

# TF-IDF + SVM
tfidf = TfidfVectorizer(max_features=5000)
X_train1_tfidf = tfidf.fit_transform(X_train1)
X_test1_tfidf = tfidf.transform(X_test1)

svm_model = SVC()
svm_model.fit(X_train1_tfidf, y_train1)

y_pred1 = svm_model.predict(X_test1_tfidf)

report = classification_report(y_test1, y_pred1, target_names=le.classes_, output_dict=True)

report_df = pd.DataFrame(report).transpose()

for col in ['precision', 'recall', 'f1-score']:
  report_df[col] = (report_df[col]*100).round(2).astype(str) + '%'

print("Skema 1: SVM + TF-IDF + 70:30")
print(report_df[['precision', 'recall', 'f1-score']])

Skema 1: SVM + TF-IDF + 70:30
             precision  recall f1-score
negative        89.94%  92.35%   91.13%
positive        90.26%  87.28%   88.75%
accuracy        90.08%  90.08%   90.08%
macro avg        90.1%  89.82%   89.94%
weighted avg    90.08%  90.08%   90.06%


## Skema 3

In [50]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(
    clean_data[text_col], clean_data['label_encoded'], test_size=0.3, random_state=42, stratify=clean_data['label_encoded'])

tfidf = TfidfVectorizer(max_features=5000)
X_train3_tfidf = tfidf.fit_transform(X_train3)
X_test3_tfidf = tfidf.transform(X_test3)

rf_model3 = RandomForestClassifier()
rf_model3.fit(X_train3_tfidf, y_train3)

y_pred3 = rf_model3.predict(X_test3_tfidf)

report = classification_report(y_test3, y_pred3, target_names=le.classes_, output_dict=True)

report_df = pd.DataFrame(report).transpose()

for col in ['precision', 'recall', 'f1-score']:
    report_df[col] = (report_df[col] * 100).round(2).astype(str) + '%'

print("Skema 3: Random Forest + TF-IDF + 70:30")
print(report_df[['precision', 'recall', 'f1-score']])

Skema 3: Random Forest + TF-IDF + 70:30
             precision  recall f1-score
negative        82.13%  90.04%    85.9%
positive        86.08%  75.87%   80.65%
accuracy        83.69%  83.69%   83.69%
macro avg        84.1%  82.95%   83.28%
weighted avg     83.9%  83.69%   83.55%
